

# Add weather data to the game results
1. Read the CSV file into a dataframe
2. Add columns for weather data and odds
3. Get latitude and longitude from stadium column
4. Use latitude and longitude to call NOAA API for weather on game date
5. Write out the enriched dataframe to a new csv file

### Import necessary libraries

In [1]:
# Libraries for formatting in/out
import json
import csv
from datetime import datetime, timedelta

# Libraries to help with data manipulation
import pandas as pd
import matplotlib.pyplot as plt

# Import Meteostat library and dependencies
!pip install meteostat
#from meteostat import Point, Daily
from meteostat import Point, Daily


# Supress Warnings Messages
import warnings
warnings.filterwarnings('ignore')


Mount Google Drive and Load the CSV to a Dataframe

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_json('/content/drive/MyDrive/Colab Notebooks/IIT/ncaa_football/1 Make CSVs/csv_files/2013-2023-json.json')

In [4]:
# Add columns for weather
df['StadiumIDNumber'] = None
df['StadiumName'] = None
df['latitude'] = None
df['longitude'] = None
df['avg_temp'] = None
df['min_temp'] = None
df['max_temp'] = None
df['prcp'] = None
df['snow'] = None
df['wspd'] = None
df['hdir'] = None
df['pres'] = None
df['tsun'] = None

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9707 entries, 0 to 9706
Data columns (total 61 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   GameID                 9707 non-null   int64         
 1   Season                 9707 non-null   int64         
 2   SeasonType             9707 non-null   int64         
 3   Week                   9707 non-null   int64         
 4   Status                 9707 non-null   object        
 5   Day                    9707 non-null   object        
 6   DateTime               9032 non-null   datetime64[ns]
 7   AwayTeam               9707 non-null   object        
 8   HomeTeam               9707 non-null   object        
 9   AwayTeamID             9707 non-null   int64         
 10  HomeTeamID             9707 non-null   int64         
 11  AwayTeamName           9707 non-null   object        
 12  HomeTeamName           9707 non-null   object        
 13  AwayTeam

In [6]:
df.describe()

,GameID,Season,SeasonType,Week,DateTime,AwayTeamID,HomeTeamID,AwayTeamScore,HomeTeamScore,TimeRemainingMinutes,...,Down,Distance,HomeRotationNumber,AwayRotationNumber,NeutralVenue,AwayPointSpreadPayout,HomePointSpreadPayout,OverPayout,UnderPayout,Attendance
count,9707.000000,9707.000000,9707.0,9707.000000,9032,9707.000000,9707.000000,8889.000000,8889.000000,7.0,...,8.0,8.0,3881.000000,3881.000000,4769.000000,2570.000000,2570.000000,2570.000000,2570.000000,2572.000000
mean,8303.223756,2018.185845,1.0,7.173483,2018-11-01 10:49:02.823294720,165.862676,159.121974,25.003262,32.412420,0.0,...,0.0,0.0,31017.929915,31016.920381,0.021388,-114.126070,-114.128016,-114.441245,-114.374319,39977.174572
min,75.000000,2013.000000,1.0,1.000000,2013-08-29 18:00:00,1.000000,1.000000,0.000000,0.000000,0.0,...,0.0,0.0,32.000000,32.000000,0.000000,-205.000000,-208.000000,-199.000000,-197.000000,0.000000
25%,6711.500000,2016.000000,1.0,3.000000,2015-11-21 12:00:00,95.000000,94.000000,11.000000,16.000000,0.0,...,0.0,0.0,141.000000,141.000000,0.000000,-153.000000,-153.000000,-153.000000,-153.000000,18253.500000
50%,9255.000000,2018.000000,1.0,7.000000,2018-10-13 15:30:00,177.000000,143.000000,21.000000,28.000000,0.0,...,0.0,0.0,237.000000,236.000000,0.000000,-124.000000,-123.000000,-124.000000,-124.000000,33749.000000
75%,11967.500000,2021.000000,1.0,11.000000,2021-10-02 12:00:00,252.000000,251.000000,36.000000,46.000000,0.0,...,0.0,0.0,437.000000,436.000000,0.000000,-71.000000,-71.000000,-71.000000,-71.000000,55847.250000
max,14534.000000,2023.000000,1.0,16.000000,2023-12-09 15:00:00,1024.000000,921.000000,117.000000,131.000000,0.0,...,0.0,0.0,525517.000000,525515.000000,1.000000,142.000000,141.000000,-34.000000,56.000000,111156.000000
std,4494.441638,3.114557,0.0,4.146278,NaN,101.136628,99.954689,19.371971,22.427205,0.0,...,0.0,0.0,106125.671634,106125.592451,0.144690,49.204521,48.943334,48.014699,48.233569,27045.944405


In [7]:
#test it
i=1

#unpack the stadium dictionary
dict_stadium = df['Stadium'][i]
df['StadiumIDNumber'][i] = dict_stadium["StadiumID"]
df['StadiumName'][i] = dict_stadium["Name"]
df['latitude'][i] = dict_stadium["GeoLat"]
df['longitude'][i] = dict_stadium["GeoLong"]

In [8]:
#build the API call
dflat = df['latitude'][i]
dflong = df['longitude'][i]

In [9]:
#override for testing
end = df['DateTime'][i]
start = end - timedelta(days=1)
thepoint = Point(dflat, dflong)
data = Daily(thepoint, start, end)

In [10]:
print(start)


2013-08-29 22:00:00


In [11]:
print(end)

2013-08-30 22:00:00


In [ ]:
data = data.fetch()

In [ ]:
# loop through dataframe and pull the latitude and longitude from the embedded stadium dictionary into new columns
for i in range(len(df)):
  if df['Stadium'][i] is not None:
    dict_stadium = df['Stadium'][i]
    df['StadiumIDNumber'][i] = dict_stadium["StadiumID"]
    df['StadiumName'][i] = dict_stadium["Name"]
    df['latitude'][i] = dict_stadium["GeoLat"]
    df['longitude'][i] = dict_stadium["GeoLong"]
    if df['DateTime'][i] is not None:
      dflat = df['latitude'][i]
      dflong = df['longitude'][i]
#     Build the Weather API Call
      end = df['DateTime'][i]
      start = end - timedelta(days=1)
      thepoint = Point(dflat, dflong)
      data = Daily(thepoint, start, end)
      data = data.fetch()
      df['avg_temp'][i] = data['tavg']
      df['min_temp'][i] = data['tmin']
      df['max_temp'][i] = data['tmax']
      df['prcp'][i] = data['prcp']
      df['snow'][i] = data['snow']
      df['wspd'][i] = data['wspd']
      df['hdir'][i] = data['wdir']
      df['pres'][i] = data['pres']
      df['tsun'][i] = data['tsun']
  if i % 100 == 0:        # Check if i is divisible by 100
    print(i), " rows processed"
#  if  20:
#    break

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400


TypeError: loop of ufunc does not support argument 0 of type NoneType which has no callable deg2rad method

In [ ]:
# copying the data to another variable to avoid any changes to original data
dfcopy = df.copy()

In [ ]:
# Write the df to a CSV file
dfcopy.to_csv("/content/drive/MyDrive/Colab Notebooks/IIT/ncaa_football/1 Make CSVs/csv_files/2013-2023-with-weather-new-csv.csv", index=False)

In [ ]:
# Write the df to a CSV file
dfcopy.to_json("/content/drive/MyDrive/Colab Notebooks/IIT/ncaa_football/1 Make CSVs/csv_files/2013-2023-with-weather-new-json.csv", orient='split', index=False)

***